In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import required libraries
import pandas as pd
from dash import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
jupyter_dash.default_mode="external"

In [5]:
#file to work with
spacex_df = pd.read_csv("~/Downloads/spacex_launch_dash.csv")

In [6]:

# Read the airline data into pandas dataframe

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    id='site_dropdown',
                                    options=[{'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}, {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                value='ALL', placeholder='Select a Launch Site', searchable=True),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={0: '0',1000: '1000',2000: '2000',3000: '3000', 4000: '4000',5000: '5000',6000: '6000',7000: '7000',8000: '8000',9000: '9000',10000: '10000'},
                                        value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site_dropdown', component_property='value'))
def select(input):
    if input == 'All Sites':
        selcted_df = spacex_df.groupby(['Launch Site'])["class"].sum().to_frame()
        selected_df = selected_df.reset_index()
        fig = px.pie(selected_df, values='class', names='Launch Site', title='Successful launches by site')
    
    else:
        selected_df = spacex_df[spacex_df["Launch Site"] == input]["class"].value_counts().to_frame()
        selected_df["name"] = ["Failure", "Success"]
        fig = px.pie(selected_df, values='class', names='name', title='Successful launches by site ' + input)

    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value"))

def get_scatter_chart(entered_site, slider_range):
    low, high = slider_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    filtered_df1 = spacex_df[mask]
    if entered_site =='ALL':
        fig = px.scatter(filtered_df1, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Correlation of Payload and Successful Missions for All Sites')
        return fig
    else:
        filtered_df2= filtered_df1[filtered_df1['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df2, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title=f'Correlation of Payload and Successful Missions for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash app running on http://127.0.0.1:8050/
